# ROBOT FIGHT SIMULATOR

22/11/2020
Jean-François BENHAIM



In [8]:
import random
import operator
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import time
import numpy as np

# Parent class Robot
class Robot:
    def __init__(self,position,direction,wslot,mvt,bodies,weapons,equipped_W,team_nb,team_name,behavior='Random',Destroyed=False):
        self.position=position
        self.direction=direction
        self.wslot=wslot
        self.mvt=mvt
        self.bodies=bodies
        self.weapons=weapons        
        self.equipped_W=equipped_W
        self.team_nb=team_nb
        self.team_name=team_name
        self.behavior=behavior
        self.Destroyed=Destroyed
        
    
    def move(self):
        global size_board
        global Change_dir
        # Robot moves 1 cell for each of its movements
        for i in self.mvt:            
            if i=="U" :
                if self.position[0]-1>=0: 
                    if board_pos[self.position[0]-1,self.position[1]]==0:
                        board_pos[self.position[0]-1,self.position[1]]=self.team_nb
                        board_pos[self.position[0],self.position[1]]=0
                        self.position[0]-=1
                        self.pick_up()
            elif i=='D':
                if self.position[0]+1<size_board:
                    if board_pos[self.position[0]+1,self.position[1]]==0:
                        board_pos[self.position[0]+1,self.position[1]]=self.team_nb
                        board_pos[self.position[0],self.position[1]]=0
                        self.position[0]+=1
                        self.pick_up()  
            elif i=='L' :
                if self.position[1]-1>=0:                     
                    if board_pos[self.position[0],self.position[1]-1]==0:
                        board_pos[self.position[0],self.position[1]-1]=self.team_nb
                        board_pos[self.position[0],self.position[1]]=0
                        self.position[1]-=1
                        self.pick_up()
            elif i=='R':
                if self.position[1]+1<size_board:                    
                    if board_pos[self.position[0],self.position[1]+1]==0:
                        board_pos[self.position[0],self.position[1]+1]=self.team_nb
                        board_pos[self.position[0],self.position[1]]=0
                        self.position[1]+=1
                        self.pick_up()                    
                
        if Change_dir.get():
            if self.position[0]==0 and self.mvt[0]=='U': self.mvt=[random.choice(['R','L','D'])]
            elif self.position[0]==size_board-1 and self.mvt[0]=='D':self.mvt=[random.choice(['R','L','U'])]
            elif self.position[1]==0 and self.mvt[0]=='L':self.mvt=[random.choice(['R','U','D'])] 
            elif self.position[1]==size_board-1 and self.mvt[0]=='R': self.mvt=[random.choice(['L','U','D'])]               

        if Smart_fire.get(): # don't fire outside the board
            if self.position[0]==0 and self.position[1]==0 : self.direction= random.choice(["D","R"])
            elif self.position[0]==size_board-1 and self.position[1]==0: self.direction=random.choice(['U','R']) 
            elif self.position[0]==0 and self.position[1]==size_board-1: self.direction=random.choice(['D','L']) 
            elif self.position[0]==size_board-1 and self.position[1]==size_board-1: self.direction=random.choice(['U','L']) 
            elif self.position[0]==0 : self.direction=random.choice(['D','R','L']) 
            elif self.position[0]==size_board-1: self.direction=random.choice(['U','L','R']) 
            elif self.position[1]==0: self.direction=random.choice(['D','U','R']) 
            elif self.position[1]==size_board-1:self.direction=random.choice(['D','U','L'])
            else: self.direction=random.choice(dir_mov_list)
        else:
            self.direction=random.choice(dir_mov_list)
        
    def pick_up(self):
        idx_obj=item_pos[self.position[0],self.position[1]]
        if idx_obj in dict_B:
            if self.behavior in ['Random','Aggressive','Defensive']:     # Pick up only if not already in the list           
                if idx_obj not in [b.index for b in self.bodies]:
                    self.bodies.append(self.equip_body(idx_obj))
                    item_pos[self.position[0],self.position[1]]=0 
            elif self.behavior=='Survivor':                             # Pick up all bodies   
                self.bodies.append(self.equip_body(idx_obj))
                item_pos[self.position[0],self.position[1]]=0 
                   
        if idx_obj in dict_W:
            w=dict_W[idx_obj]
            if self.behavior in ['Random','Defensive']:                 # Pick up only if not already in the list    
                if w.index not in self.weapons:
                    self.weapons.append(w.index)
                    item_pos[self.position[0],self.position[1]]=0
            if self.behavior in ['Aggressive','Survivor']:              # Pick up all weapon except Basic shoot
                if w.index>1:
                    self.weapons.append(w.index)
                    item_pos[self.position[0],self.position[1]]=0


    def attack(self):
        affected_cells=[]
        print_att=[]
        for att in self.equipped_W:
            print_att.append(att.name)           
            if att.index!=6: # the Twin weapon only adds weapons slot and does not have any damaging effect                
                for cell in att.effect(self.direction,self.position):
                    affected_cells.append(cell) 
        print('attack',self.team_name,self.position,print_att,affected_cells)
                   
        return affected_cells

    def rob_damage(self,dmg):
        before=self.bodies[0].HP
        self.bodies[0].HP-=dmg
        print('damage',self.team_name,self.position,self.bodies[0].name,'B:',before,'A:',self.bodies[0].HP)
        if self.bodies[0].HP<=0:
            print('DEAD', self.team_name,self.position)
            self.bodies[0].HP=0
            self.Destroyed=True
            print('destroyed?',self.Destroyed,'destroyed_body',self.bodies[0])
            board_pos[self.position[0],self.position[1]]=0
            item_pos[self.position[0],self.position[1]]=random.choice(items_idx)
    
    def choose_W_B(self):
        if self.behavior== 'Random':
            # Management of Bodies
            random.shuffle(self.bodies)
            # if equipped body has an additional movement, it is added. If not, only the first movement is kept
            if self.bodies[0].mvt==1:
                if  len(self.mvt)==1: self.mvt.append(random.choice(dir_mov_list))
                else: self.mvt=[self.mvt[0]]               
            self.wslot=1+self.bodies[0].wslot
            # Management of the weapons                  
            self.equipped_W=[]
            w_count=0
            random.shuffle(self.weapons) 
            while w_count<min(self.wslot,len(self.weapons)): 
                new_W=self.weapons[w_count]
                self.equipped_W.append(dict_W[new_W])
                # if Twin Weapon is equipped, two weapon slot are added
                if new_W==6: self.wslot+=2
                w_count+=1
                

        # AGGRESSIVE Robots always try to equip Battle Bodies 
        # For Weapons it will equip Twin weapons in priority then the highest indexes available Weapons (Weapons considered to be the best have a higher index)
        if self.behavior=='Aggressive':
            # Management of Bodies
            if 13 in [b.index for b in self.bodies]:
                p=[i for i, j in enumerate(self.bodies) if j.index == 13]
                self.bodies[0],self.bodies[p[0]]=self.bodies[p[0]],self.bodies[0]
                self.mvt=[self.mvt[0]]                
            else:
                random.shuffle(self.bodies)            
                if self.bodies[0].mvt==1:
                    if  len(self.mvt)==1: self.smart_new_move() 
                else: self.mvt=[self.mvt[0]]   
            self.wslot=1+self.bodies[0].wslot
            
            # Management of the weapons
            self.equipped_W=[]            
            self.weapons.sort()
            twin_count=[i for i, j in enumerate(self.weapons) if j == 6]
            if twin_count!=[]:
                for i in twin_count:
                    self.wslot+=2 
                    self.equipped_W.append(dict_W[self.weapons[i]]) 
            for i in range(min(self.wslot,len(self.weapons))-len(twin_count)):
                new_W=dict_W[self.weapons[len(self.weapons)-len(twin_count)-i-1]]
                self.equipped_W.append(new_W)
            
        # DEFENSIVE Robots always try to equip Hard Bodies and explosion and will use the highest available weapon if not
        if self.behavior=="Defensive":
            # Management of Bodies
            if 11 in [b.index for b in self.bodies]:
                p=[i for i, j in enumerate(self.bodies) if j.index == 11]
                self.bodies[0],self.bodies[p[0]]=self.bodies[p[0]],self.bodies[0]                
            else:
                random.shuffle(self.bodies)            
                if self.bodies[0].mvt==1:
                    if  len(self.mvt)==1: self.smart_new_move() 
                else: self.mvt=[self.mvt[0]]    
            self.wslot=1+self.bodies[0].wslot                     
            # Management of the weapons
            self.equipped_W=[]
            self.weapons.sort()
            w_count=0
            if 4 in self.weapons:
                self.equipped_W.append(Explosion())
                w_count+=1
            while w_count<min(self.wslot,len(self.weapons)):  
                self.equipped_W.append(dict_W[self.weapons[len(self.weapons)-w_count-1]])
                if self.weapons[len(self.weapons)-w_count-1]==6: self.wslot+=2
                w_count+=1
            
        
        # SURVIVOR ROBOTS Will always equip the body with highest HP and have the same behaviour as Offensive for weapons
        if self.behavior=="Survivor":
            # Management of Bodies
            self.bodies=sorted(self.bodies,key=operator.attrgetter("HP"),reverse=True)
            self.wslot=1+self.bodies[0].wslot
            if self.bodies[0].mvt==1:
                if  len(self.mvt)==1: self.smart_new_move() 
            else: self.mvt=[self.mvt[0]]                      
            # Management of the weapons
            self.equipped_W=[]            
            self.weapons.sort()
            twin_count=[i for i, j in enumerate(self.weapons) if j == 6]
            # print("twin count Sur",twin_count)
            if twin_count!=[]:
                for i in twin_count:
                    self.wslot+=2 
                    self.equipped_W.append(dict_W[self.weapons[i]]) 
            for i in range(min(self.wslot,len(self.weapons))-len(twin_count)):
                new_W=dict_W[self.weapons[len(self.weapons)-len(twin_count)-i-1]]
                self.equipped_W.append(new_W)

    def smart_new_move(self):
        if self.mvt[0]=='U': self.mvt.append(random.choice(['U','L','R']))
        if self.mvt[0]=='D': self.mvt.append(random.choice(['D','L','R']))
        if self.mvt[0]=='R': self.mvt.append(random.choice(['U','D','R']))
        if self.mvt[0]=='L': self.mvt.append(random.choice(['U','L','D']))

    def equip_body(self,idx_b):
        if idx_b==10: return Simple_body()
        elif idx_b==11: return Hard_body()
        elif idx_b==12: return Light_body()
        elif idx_b==13: return Battle_body()
        elif idx_b==20: return deact_body()

class Deact_robot(Robot):
    def __init__(self,position,direction):
        super().__init__(position,direction,1,[],[deact_body()],[Basic_shoot()],[Basic_shoot()],3,"Grey")
        self.position=position
        self.direction=direction
    
    def move(self):
        global size_board
        if Smart_fire.get(): # don't fire outside the board
            
            if self.position[0]==0 and self.position[1]==0 : self.direction= random.choice(["D","R"])
            elif self.position[0]==size_board-1 and self.position[1]==0: self.direction=random.choice(['U','R']) 
            elif self.position[0]==0 and self.position[1]==size_board-1: self.direction=random.choice(['D','L']) 
            elif self.position[0]==size_board-1 and self.position[1]==size_board-1: self.direction=random.choice(['U','L']) 
            elif self.position[0]==0 : self.direction=random.choice(['D','R','L']) 
            elif self.position[0]==size_board-1: self.direction=random.choice(['U','L','R']) 
            elif self.position[1]==0: self.direction=random.choice(['D','U','R']) 
            elif self.position[1]==size_board-1:self.direction=random.choice(['D','U','L'])
            else: self.direction=random.choice(dir_mov_list)
        else:            
            self.direction=random.choice(dir_mov_list)

    def choose_W_B(self):
        pass


########### Parent class BODIES #########################
class Bodies:
    def __init__(self,HP,mvt,wslot,name,index):
        self.HP=HP
        self.mvt=mvt
        self.wslot=wslot
        self.name=name
        self.index=index
# All body classes inherited from Bodies class
class Simple_body(Bodies):
    def __init__(self):
        super().__init__(2,0,0,"Simple Body",10)

class Hard_body(Bodies):
    def __init__(self):
        super().__init__(5,0,0,"Hard Body",11)

class Light_body(Bodies):
    def __init__(self):
        super().__init__(3,1,0,"Light Body",12)

class Battle_body(Bodies):
    def __init__(self):
        super().__init__(2,0,1,"Battle Body",13)

class deact_body(Bodies):
    def __init__(self):
        super().__init__(1,0,0,"Deactivated Robot Body",20)


########### Parent class WEAPONS ######################
class Weapons:
    def __init__(self,index,name,damage):
        self.index=index
        self.name=name
        self.damage=damage
      
        
# All weapon classes inherited from Weapons class
class Basic_shoot(Weapons):
    def __init__(self):        
        super().__init__(1,"Basic Shot",1)
    
    def effect(self,direction,position):
        affected_cells=[]
        if direction=='U':
            for i in range(1,3):
                if position[0]-i>=0:
                    affected_cells.append(((position[0]-i,position[1]),self.damage))
        if direction=='D':
            for i in range(1,3):
                if position[0]+i<size_board:
                    affected_cells.append(((position[0]+i,position[1]),self.damage))
        if direction=='L':
            for i in range(1,3):
                if position[1]-i>=0:
                    affected_cells.append(((position[0],position[1]-i),self.damage))
        if direction=='R':
            for i in range(1,3):
                if position[1]+i<size_board:
                    affected_cells.append(((position[0],position[1]+i),self.damage))
        return affected_cells
        
class Laser(Weapons):
    def __init__(self):
        super().__init__(2,"Laser",1)
    
    def effect(self,direction,position):
        affected_cells=[]
        if direction=='U':
            for i in range(1,size_board):
                if position[0]-i>=0:
                    affected_cells.append(((position[0]-i,position[1]),self.damage))
        if direction=='D':
            for i in range(1,size_board):
                if position[0]+i<size_board:
                    affected_cells.append(((position[0]+i,position[1]),self.damage))
        if direction=='L':
            for i in range(1,size_board):
                if position[1]-i>=0:
                    affected_cells.append(((position[0],position[1]-i),self.damage))
        if direction=='R':
            for i in range(1,size_board):
                if position[1]+i<size_board:
                    affected_cells.append(((position[0],position[1]+i),self.damage))
        return affected_cells

class Sword(Weapons):
    def __init__(self):
        super().__init__(3,"Sword",2)
    
    def effect(self,direction,position):
        affected_cells=[]
        if direction=='U':
            if position[0]-1>=0:
                for i in range(-1,2):                  
                    if position[1]+i<size_board and position[1]+i>=0: affected_cells.append(((position[0]-1,position[1]+i),self.damage))
        if direction=='D':
            if position[0]+1<size_board:
                for i in range(-1,2):                  
                    if position[1]+i<size_board and position[1]+i>=0: affected_cells.append(((position[0]+1,position[1]+i),self.damage))
        if direction=='L':
            if position[1]-1>=0:
                for i in range(-1,2):                  
                    if position[0]+i<size_board and position[0]+i>=0: affected_cells.append(((position[0]+i,position[1]-1),self.damage))
        if direction=='R':
            if position[0]+1<size_board:
                for i in range(-1,2):                  
                    if position[0]+i<size_board and position[0]+i>=0: affected_cells.append(((position[0]+i,position[1]+1),self.damage))
        return affected_cells

class Explosion(Weapons):
    def __init__(self):
        super().__init__(4,"Explosion",1)
    
    def effect(self,direction,position):
        affected_cells=[]
        for i in range(-1,2):
            for j in range(-1,2):
                if position[0]+i<size_board and position[0]+i>=0 and position[1]+j<size_board and position[1]+j>=0 and (j!=0 or i!=0):
                    affected_cells.append(((position[0]+i,position[1]+j),self.damage))
        return affected_cells

class Dual_Laser(Weapons):
    def __init__(self):
        super().__init__(5,"Dual Laser",1)

    def effect(self,direction,position):
        affected_cells=[]
        if direction=='U' or direction=='D':
            for i in range(-size_board+1,size_board):
                if position[0]+i>=0 and position[0]+i<size_board and i!=0:
                    affected_cells.append(((position[0]+i,position[1]),self.damage))
        if direction=='L' or direction=='R':
            for i in range(-size_board+1,size_board):
                if position[1]+i>=0 and position[1]+i<size_board and i!=0:
                    affected_cells.append(((position[0],position[1]+i),self.damage))
        
        return affected_cells

class Twin_Weapon(Weapons):
    def __init__(self):
        super().__init__(6,"Twin Weapon",0)

In [9]:
# CODE to have text displayed when hovering over a widget. I DID NOT CODE THAT PART
# Coming from https://stackoverflow.com/questions/20399243/display-message-when-hovering-over-something-with-mouse-cursor-in-python

#### ORIGINAL TOOL TIP
class ToolTip1(object):

    def __init__(self, widget):
        self.widget = widget
        self.tipwindow = None
        self.id = None
        self.x = self.y = 0

    def showtip(self, text):
        "Display text in tooltip window"
        self.text = text
        if self.tipwindow or not self.text:
            return
        x, y, cx, cy = self.widget.bbox("insert")
        x = x + self.widget.winfo_rootx() + 57
        y = y + cy + self.widget.winfo_rooty() +27
        self.tipwindow = tw = tk.Toplevel(self.widget)
        tw.wm_overrideredirect(1)
        tw.wm_geometry("+%d+%d" % (x, y))
        label = tk.Label(tw, text=self.text, justify=tk.LEFT,
                      background="#ffffe0", relief=tk.SOLID, borderwidth=1,
                      font=("tahoma", "8", "normal"))
        label.pack(ipadx=1)

    def hidetip(self):
        tw = self.tipwindow
        self.tipwindow = None
        if tw:
            tw.destroy()

def CreateToolTip(widget, text):
    toolTip = ToolTip1(widget)
    def enter(event):
        toolTip.showtip(text)
    def leave(event):
        toolTip.hidetip()
    widget.bind('<Enter>', enter)
    widget.bind('<Leave>', leave)
# END of code from STACKOVERFLOW link

In [25]:


# Create a dictionary of all WEAPONS based on their index
def dict_weapons():
    dict={}
    w_id=[w().index for w in Weapons.__subclasses__()]
    w_obj=[w() for w in Weapons.__subclasses__()]
    for i in range(len(w_obj)):
        dict[w_id[i]]=w_obj[i]
    return dict

# Create a dictionary of all BODIES based on their index
def dict_bodies():
    dict={}
    b_id=[b().index for b in Bodies.__subclasses__()]
    b_obj=[b() for b in Bodies.__subclasses__()]
    for i in range(len(b_obj)):
        dict[b_id[i]]=b_obj[i]
    return dict

      

dict_B=dict_bodies()
dict_W=dict_weapons()
items_idx=[*dict_W]+[*dict_B] 
items_idx.remove(20) # Deactivated body not available for pick up
# items_idx=[12]
dir_mov_list=['U','D','L','R']

# Load all illustrations in PIL format image
R_rob_U=Image.open("Images/Red_up.png")
R_rob_U=R_rob_U.resize((35,35),Image.ANTIALIAS)
R_rob_D=R_rob_U.rotate(180)
R_rob_L=R_rob_U.rotate(90)
R_rob_R=R_rob_U.rotate(270)
Img_red={'U':R_rob_U,'D':R_rob_D,'L':R_rob_L,'R':R_rob_R}
B_rob_U=Image.open("Images/Blue_up.png")
B_rob_U=B_rob_U.resize((35,35),Image.ANTIALIAS)
B_rob_D=B_rob_U.rotate(180)
B_rob_L=B_rob_U.rotate(90)
B_rob_R=B_rob_U.rotate(270)
Img_blue={'U':B_rob_U,'D':B_rob_D,'L':B_rob_L,'R':B_rob_R}
G_rob_img=Image.open("Images/Grey_up.png")
G_rob_img=G_rob_img.resize((35,35),Image.ANTIALIAS)
W_img=Image.open("Images/Weapon.png")
W_img=W_img.resize((25,25),Image.ANTIALIAS)
B_img=Image.open("Images/Body.png")
B_img=B_img.resize((25,25),Image.ANTIALIAS)






class MainApplication(tk.Frame):
    
    
    def __init__(self, master):
        global size_board
        global Friendly_fire
        global Smart_fire
        global Grey_fire
        global Change_dir
        size_board=6
        self.master = master
        tk.Frame.__init__(self, self.master)
        self.Size_b = tk.IntVar()
        self.Size_b.set(1) 
        self.Add_Deact=tk.BooleanVar()
        self.Add_Deact.set(False)  
        Friendly_fire=tk.BooleanVar()
        Friendly_fire.set(False) 
        Smart_fire=tk.BooleanVar()
        Smart_fire.set(False)
        Grey_fire=tk.BooleanVar()
        Grey_fire.set(False)
        Change_dir=tk.BooleanVar()
        Change_dir.set(False)     
        self.configure_gui()
        self.create_widgets()
        
        
   
    def configure_gui(self): 
        self.master.title("Robot Fight Simulator")
        self.window_size=['500x550','600x650','700x750']        
        self.master.geometry(self.window_size[0])
        self.master.resizable(False, False)
        self.master.grid_rowconfigure(0, weight=1)
        self.master.grid_columnconfigure(0, weight=1)        
        

# Function creates the empty board n*n size       
    def create_widgets(self):       
        global cells
        cells=[]
        # Create frames to display board in the top frame then select the number of robots of each team below the board and finally a frame that will appear later to manage the turns
        self.Frame_grid=tk.Frame(self.master)
        self.Frame_grid.grid(row=0,sticky=tk.N,pady=10)
        self.Frame_team=tk.Frame(self.master)
        self.Frame_team.grid(row=1)
        self.Frame_turn=tk.Frame(self.master)
        self.Frame_team.grid_rowconfigure(0, weight=1)
        self.Frame_turn.grid_rowconfigure(0, weight=1)
        self.Frame_team.grid_columnconfigure(0, weight=1)  
        self.Frame_turn.grid_columnconfigure(0, weight=1)  
        
        # Create the board
        count=0
        img=tk.PhotoImage(width=1, height=1)   
        for i in range(size_board):
            for j in range(size_board):
                     
                cells.append(tk.Button(self.Frame_grid,image=img,bg='white',height=40,width=40,padx=0,pady=0))    
                cells[count].grid(column=j, row=i)                         
                count+=1
        for i in range (size_board):
            cells[i].configure(bg="#02C6FD")
        for i in range (size_board):
            cells[(size_board-1)*size_board+i].configure(bg="#FF8F8F")

        # Create the buttons & Label on Frame_team
           
        L_Teams=tk.Label(self.Frame_team,text="Select Number of robots per team", font=("Calibri",14))
        L_Teams.grid(row=0,column=0,columnspan=4,pady=10, sticky=tk.W+tk.E+tk.N+tk.S)
        L_red_nb=tk.Label(self.Frame_team,text="Red Team:",fg='red',font=('Calibri bold',12))
        L_red_nb.grid(row=1,column=0,sticky=tk.N)
        L_blue_nb=tk.Label(self.Frame_team,text="Blue Team:",fg='blue',font=('Calibri bold',12))
        L_blue_nb.grid(row=1,column=2, sticky=tk.N)
        L_Random=tk.Label(self.Frame_team,text="Random",font=('Calibri',12))
        L_Random.grid(row=2,column=1,padx=5,pady=2, sticky=tk.N)
        L_Agg=tk.Label(self.Frame_team,text="Aggressive",font=('Calibri',12))
        L_Agg.grid(row=3,column=1,padx=5,pady=2, sticky=tk.N)

        L_Def=tk.Label(self.Frame_team,text="Defensive",font=('Calibri',12))
        L_Def.grid(row=4,column=1,padx=5,pady=2, sticky=tk.N)
        L_Surv=tk.Label(self.Frame_team,text="Survivor",font=('Calibri',12))
        L_Surv.grid(row=5,column=1,padx=5,pady=2, sticky=tk.N)
        self.Red_rand=tk.Spinbox(self.Frame_team,from_=0, to_=size_board,width=10,justify='center')
        self.Red_rand.delete(0,'end')
        self.Red_rand.insert(0,size_board//2)
        self.Red_rand.grid(row=2,column=0,pady=2)
        self.Red_Agg=tk.Spinbox(self.Frame_team,from_=0, to_=size_board,width=10,justify='center')        
        self.Red_Agg.grid(row=3,column=0,pady=2)
        self.Red_Def=tk.Spinbox(self.Frame_team,from_=0, to_=size_board,width=10,justify='center')        
        self.Red_Def.grid(row=4,column=0,pady=2)
        self.Red_Surv=tk.Spinbox(self.Frame_team,from_=0, to_=size_board,width=10,justify='center')        
        self.Red_Surv.grid(row=5,column=0,pady=2)
        self.Blue_rand=tk.Spinbox(self.Frame_team,from_=0, to_=size_board,width=10,justify='center')
        self.Blue_rand.delete(0,'end')
        self.Blue_rand.insert(0,size_board//2)        
        self.Blue_rand.grid(row=2,column=2,padx=10,pady=2)
        self.Blue_Agg=tk.Spinbox(self.Frame_team,from_=0, to_=size_board,width=10,justify='center')        
        self.Blue_Agg.grid(row=3,column=2,pady=2)
        self.Blue_Def=tk.Spinbox(self.Frame_team,from_=0, to_=size_board,width=10,justify='center')        
        self.Blue_Def.grid(row=4,column=2,pady=2)
        self.Blue_Surv=tk.Spinbox(self.Frame_team,from_=0, to_=size_board,width=10,justify='center')        
        self.Blue_Surv.grid(row=5,column=2,pady=2)
        Fill=tk.Button(self.Frame_team,text="Fill grid",font=("Calibri",12),command=self.Fill_board)
        Fill.grid(column=0,row=6,columnspan=4,pady=10, sticky=tk.N)

        # Create the buttons & Label on Frame_Turn
        self.L_score_R=tk.Label(self.Frame_turn,fg='red',font=("Calibri",16))
        self.L_score_R.grid(column=0,row=0,columnspan=2,pady=4, sticky=tk.N)
        self.L_score_B=tk.Label(self.Frame_turn,fg='blue',font=("Calibri",16))
        self.L_score_B.grid(column=0,row=1,columnspan=2,pady=4, sticky=tk.N)
        self.Turn=tk.Button(self.Frame_turn,text="Next turn",font=("Calibri",12),command=self.next_T)
        self.Turn.grid(column=0,row=2,columnspan=2,pady=10, sticky=tk.N)
        self.Full_game=tk.Button(self.Frame_turn,text="Simulate the following \nnumber of turns",font=("Calibri",12),command=self.Complete_popup)
        self.Full_game.grid(column=0,row=3,pady=10, sticky=tk.N)
        self.nb_sim=ttk.Combobox(self.Frame_turn,values=[10,25,50,100],width=10,justify='center')
        # self.nb_sim.option_add('*TCombobox*Listbox.Justify', 'center')   
        self.nb_sim.current(0)
        self.nb_sim.grid(column=1,row=3,pady=10,padx=10,sticky=tk.W)
         
        #Create the Menu
        menu = tk.Menu(self.master)
        Game_cascade = tk.Menu(menu, tearoff="False")
        Option_cascade = tk.Menu(menu, tearoff="False")
        Option_size=tk.Menu(Option_cascade,tearoff="False")

        menu.add_cascade(label='Game', menu=Game_cascade)
        menu.add_cascade(label='Options', menu=Option_cascade)
       
        
        Game_cascade.add_command(label='New Game',command=self.new_game)
        Game_cascade.add_separator()
        Game_cascade.add_command(label='Quit',command=self.quit_game)
        Option_cascade.add_cascade(label="Select board size", menu=Option_size)
        Option_cascade.add_separator()
        Option_cascade.add_checkbutton(label='No Friendly Fire',variable=Friendly_fire)       
        Option_cascade.add_checkbutton(label='Smart Fire',variable=Smart_fire)        
        Option_cascade.add_checkbutton(label='Smart Movement',variable=Change_dir)
        Option_cascade.add_separator()
        Option_cascade.add_checkbutton(label='Deact. Robots fire',variable=Grey_fire)
        Option_cascade.add_checkbutton(label="Add React. Robots every 10 turns",variable=self.Add_Deact)
        Option_size.add_radiobutton(label='6x6', variable=self.Size_b, value=1)
        Option_size.add_radiobutton(label='8x8', variable=self.Size_b, value=2)
        Option_size.add_radiobutton(label='10x10', variable=self.Size_b, value=3)

        
        self.master.config(menu=menu)

    
# Function that display robot images on the board      
    def Fill_Robot(self):
        count=0
        
        for i in range(size_board):
            for j in range(size_board):
                IsItem=False
                if board_pos[i,j]==1:
                    robr=self.Robot_at(i,j,red_team)
                    img=ImageTk.PhotoImage(Img_red[robr[1].direction])
                    hovertext="Red{} - HP:{}\nType:{}\nB:{}\nW:{}\nMvt:{}".format(robr[0],robr[1].bodies[0].HP,robr[1].behavior,robr[1].bodies[0].name,', '.join([o.name for o in robr[1].equipped_W]),', '.join(robr[1].mvt))
                                    
                elif board_pos[i,j]==2:
                    robb=self.Robot_at(i,j,blue_team)
                    img=ImageTk.PhotoImage(Img_blue[robb[1].direction])
                    hovertext="Blue{} - HP:{}\nType:{}\nB:{}\nW:{}\nMvt:{}".format(robb[0],robb[1].bodies[0].HP,robb[1].behavior,robb[1].bodies[0].name,', '.join([o.name for o in robb[1].equipped_W]),', '.join(robb[1].mvt))
                    
                elif board_pos[i,j]==3:
                    img=ImageTk.PhotoImage(G_rob_img)
                    hovertext="Deactivated Robot"
                    
                elif item_pos[i,j] in dict_B: 
                    img=ImageTk.PhotoImage(B_img)
                    IsItem=True
                    hovertext=dict_B[item_pos[i,j]].name

                elif item_pos[i,j] in dict_W: 
                    img=ImageTk.PhotoImage(W_img)
                    IsItem=True
                    hovertext=dict_W[item_pos[i,j]].name    
                else:
                    img=tk.PhotoImage(width=1, height=1)
                    hovertext=''        
                if IsItem:
                    cells[count].configure(image=img, text=item_pos[i,j], compound='top',font=('Calibri',8))
                    CreateToolTip(widget = cells[count], text = hovertext) 
                else:
                    cells[count].configure(image=img,text='')
                    CreateToolTip(widget = cells[count], text = hovertext) 
                cells[count].image=img
                count+=1
               

    def Fill_board(self):
        self.Red_nb=int(self.Red_rand.get())+int(self.Red_Agg.get())+int(self.Red_Def.get())+int(self.Red_Surv.get())
        self.Blue_nb=int(self.Blue_rand.get())+int(self.Blue_Agg.get())+int(self.Blue_Def.get())+int(self.Blue_Surv.get())
        if self.Red_nb==0: 
            self.wrong_quantity(False,'Red Team')
            return
        if self.Blue_nb==0:
            self.wrong_quantity(False,'Blue Team')
            return
        if self.Red_nb>size_board: 
            self.wrong_quantity(True,'Red Team')
            return
        if self.Blue_nb>size_board:
            self.wrong_quantity(True,'Blue Team')
            return
        self.initialise_game()
        self.Frame_team.grid_forget()
        self.L_score_R.configure(text="Red team: {}/{} active Robots".format(self.Red_nb,self.Red_nb))
        self.L_score_B.configure(text="Blue team: {}/{} active Robots".format(self.Blue_nb,self.Blue_nb))
        self.Frame_turn.grid(row=2,sticky=tk.N)        
        self.Fill_Robot()

# Functions that reinitialise the board ready for a new game
    def new_game(self):
        global size_board
        if self.Size_b.get()==1: 
            size_board=6
            window_s=self.window_size[0]            
        elif self.Size_b.get()==2: 
            size_board=8
            window_s=self.window_size[1]
        elif self.Size_b.get()==3: 
            size_board=10
            window_s=self.window_size[2]
        
        self.Frame_team.destroy()
        self.Frame_grid.destroy()
        self.Frame_turn.destroy()
        self.master.geometry(window_s)
        self.create_widgets()

    def quit_game(self):
        self.master.destroy()


### ALL functions managing the game LOGIC
# Function that initialise the board and position all robots
# It takes as argument the number of robots requested by the player on each team as well as the size of the board
# It also fill positions of each robot in a numpy array "board_pos" where each robot position is represented by a number (1:red team, 2:blue team, 3: deactivated robots)

    def initialise_game(self):
        global red_team,blue_team,grey_team
        red_team,blue_team,grey_team=[],[],[]
        global board_pos,item_pos
        self.Turn_count=0
        board_pos=np.zeros((size_board,size_board),dtype='int32')
        item_pos=np.zeros((size_board,size_board),dtype='int32')        
        # assign random positions to the robots of each team
        positions_red=random.sample(range(size_board),self.Red_nb)
        positions_blue=random.sample(range(size_board),self.Blue_nb)
        # Create robots of blue and red teams
        pos_r,pos_b=0,0
        for i in range(int(self.Red_rand.get())):       
            red_team.append(Robot([size_board-1,positions_red[pos_r]],'U',1,[random.choice(['U','L','R'])],[Simple_body()],[1],[Basic_shoot()],1,"Red"))
            board_pos[size_board-1,positions_red[pos_r]]=1
            pos_r+=1
        for i in range(int(self.Red_Agg.get())): 
            red_team.append(Robot([size_board-1,positions_red[pos_r]],'U',1,[random.choice(['U','L','R'])],[Simple_body()],[1],[Basic_shoot()],1,"Red","Aggressive"))
            board_pos[size_board-1,positions_red[pos_r]]=1
            pos_r+=1
        for i in range(int(self.Red_Def.get())): 
            red_team.append(Robot([size_board-1,positions_red[pos_r]],'U',1,[random.choice(['U','L','R'])],[Simple_body()],[1],[Basic_shoot()],1,"Red","Defensive"))
            board_pos[size_board-1,positions_red[pos_r]]=1
            pos_r+=1 
        for i in range(int(self.Red_Surv.get())): 
            red_team.append(Robot([size_board-1,positions_red[pos_r]],'U',1,[random.choice(['U','L','R'])],[Simple_body()],[1],[Basic_shoot()],1,"Red","Survivor"))
            board_pos[size_board-1,positions_red[pos_r]]=1
            pos_r+=1           
        for j in range(int(self.Blue_rand.get())):
            blue_team.append(Robot([0,positions_blue[pos_b]],'D',1,[random.choice(['D','L','R'])],[Simple_body()],[1],[Basic_shoot()],2,"Blue"))
            board_pos[0,positions_blue[pos_b]]=2
            pos_b+=1
        for j in range(int(self.Blue_Agg.get())):
            blue_team.append(Robot([0,positions_blue[pos_b]],'D',1,[random.choice(['D','L','R'])],[Simple_body()],[1],[Basic_shoot()],2,"Blue","Aggressive"))
            board_pos[0,positions_blue[pos_b]]=2
            pos_b+=1
        for j in range(int(self.Blue_Def.get())):
            blue_team.append(Robot([0,positions_blue[pos_b]],'D',1,[random.choice(['D','L','R'])],[Simple_body()],[1],[Basic_shoot()],2,"Blue","Defensive"))
            board_pos[0,positions_blue[pos_b]]=2
            pos_b+=1
        for j in range(int(self.Blue_Surv.get())):
            blue_team.append(Robot([0,positions_blue[pos_b]],'D',1,[random.choice(['D','L','R'])],[Simple_body()],[1],[Basic_shoot()],2,"Blue","Survivor"))
            board_pos[0,positions_blue[pos_b]]=2
            pos_b+=1
        # Create 8 deactivated robots ("grey team")
        open_pos_grey=[]
        for k in range(1,size_board-1):
            for l in range(size_board):
                open_pos_grey.append([k,l])
        positions_grey=random.sample(open_pos_grey,8)
        for m in range(8):
            grey_team.append(Deact_robot(positions_grey[m],random.choice(dir_mov_list)))
            board_pos[positions_grey[m][0],positions_grey[m][1]]=3

# Function that manage one turn, it takes a robot as argument and goes through all actions of a robot in one turn
    def turn(self,robot):
        img_old=tk.PhotoImage(width=1, height=1)
        if robot.team_nb!=3: 
            cells[robot.position[0]*size_board+robot.position[1]].configure(image=img_old)
            cells[robot.position[0]*size_board+robot.position[1]].image=img_old
        robot.move()
        time.sleep(0.5)
        
        if robot.team_nb==1:img=ImageTk.PhotoImage(Img_red[robot.direction])                  
        elif robot.team_nb==2:img=ImageTk.PhotoImage(Img_blue[robot.direction])
        elif robot.team_nb==3:img=ImageTk.PhotoImage(G_rob_img)                    
        else: img=tk.PhotoImage(width=1, height=1)        
        cells[robot.position[0]*size_board+robot.position[1]].configure(image=img,bg="black")
        cells[robot.position[0]*size_board+robot.position[1]].image=img        
        aff_cells=robot.attack()        
        for acell in aff_cells:
            affected_robot=None
            cel_coor=[acell[0][0],acell[0][1]]  
            cells[cel_coor[0]*size_board+cel_coor[1]].configure(bg='red')          
            if board_pos[acell[0][0],acell[0][1]]==1:
                affected_robot=next((rob for rob in red_team if rob.position == cel_coor), None)                
            elif board_pos[acell[0][0],acell[0][1]]==2:
                affected_robot=next((rob for rob in blue_team if rob.position == cel_coor), None)                
            elif board_pos[acell[0][0],acell[0][1]]==3:                
                affected_robot=next((rob for rob in grey_team if rob.position == cel_coor), None)
            # Inflict damage only if Robots are not in the same team or if Friendly fire is activated (Boolean at False)
            
            if affected_robot!=None and affected_robot.Destroyed==False and (affected_robot.team_name!=robot.team_name or not Friendly_fire.get()):
                affected_robot.rob_damage(acell[1])
        self.master.update()        
        time.sleep(0.5)
        aff_cells.append(((robot.position[0],robot.position[1]),0))
        self.Fill_Robot()  
        for acell in aff_cells:
            cel_coor=[acell[0][0],acell[0][1]]  
            if cel_coor[0]==0:cells[cel_coor[1]].configure(bg="#02C6FD")
            elif cel_coor[0]==size_board-1: cells[cel_coor[0]*size_board+cel_coor[1]].configure(bg="#FF8F8F")
            else: cells[cel_coor[0]*size_board+cel_coor[1]].configure(bg="white")
        robot.choose_W_B()

# Function that checks if a robot of a designated team is at the given position
    def Robot_at(self,row,col,team):
        return next(((i,rob) for i,rob in enumerate(team) if rob.position == [row,col]), None)

# Function that manage the sequence of turns. It keeps memory of which team needs to play next and select the next player of this team (randomly)  
# It also manages the addition of extra deactivated robots if the option is activated
    def next_T(self):
        global red_team,blue_team,grey_team
        self.Turn['state'] = tk.DISABLED
        self.Full_game['state'] = tk.DISABLED
        print('############ NEW TURN {} #######'.format(self.Turn_count))
        if len(red_team)==0 or len(blue_team)==0: return False        
        if self.Turn_count%2==0:
            self.turn(random.choice(red_team))
        else:
            self.turn(random.choice(blue_team))

        self.Turn_count+=1

        # If "Deact Robot fire" is activated, a random Deact Robot will fire
        self.master.update() 
        if Grey_fire.get():                    
            time.sleep(0.5)
            print('---> GREY ATTACK')
            grey_team=[rob for rob in grey_team if rob.Destroyed != True]
            if grey_team!=[]: self.turn(random.choice(grey_team))
            time.sleep(0.5)

        # If the "Add deactivated robots" is activated, 1 to 5 robots (random value) will be added if there is enough room on the board
        if self.Turn_count%10==0 and self.Add_Deact.get():
            
            available_cell=[]
            for i in range(size_board):
                for j in range(size_board):
                    if board_pos[i,j]==0 and item_pos[i,j]==0:
                        available_cell.append([i,j])        
            nb_new_grey=random.randrange(1,5)            
            if len(available_cell)>nb_new_grey+1:
                new_grey=random.sample(available_cell,nb_new_grey)             
                for i in new_grey:                   
                    grey_team.append(Deact_robot([i[0],i[1]],random.choice(dir_mov_list)))
                    board_pos[i[0],i[1]]=3
        print('end of turn')    
        self.Fill_Robot()
        # error_HP_red=[(rob.bodies,rob.bodies[0].HP,rob.position) for rob in red_team if rob.Destroyed==False and rob.bodies[0].HP<=0]
        # error_HP_blue=[(rob.bodies,rob.bodies[0].HP,rob.position) for rob in blue_team if rob.Destroyed==False and rob.bodies[0].HP<=0]
        # error_HP_grey=[(rob.bodies,rob.bodies[0].HP,rob.position) for rob in grey_team if rob.Destroyed==False and rob.bodies[0].HP<=0]
        # print('Red', error_HP_red,'Blue',error_HP_blue,'grey',error_HP_grey)
        # if not( error_HP_blue==[] and error_HP_red==[]): return 'WTF'
        red_team=[rob for rob in red_team if rob.Destroyed != True]
        blue_team=[rob for rob in blue_team if rob.Destroyed != True]
        self.L_score_R.configure(text="Red team: {}/{} active Robots".format(len(red_team),self.Red_nb))
        self.L_score_B.configure(text="Blue team: {}/{} active Robots".format(len(blue_team),self.Blue_nb))
        if len(blue_team)==0:
            self.Victory_popup('Red Team',"Blue Team")
        elif len(red_team)==0:
            self.Victory_popup('Blue Team',"Red Team")
        print('################# END of TURN {} ############'.format(self.Turn_count-1))
        self.Turn['state'] = tk.NORMAL
        self.Full_game['state'] = tk.NORMAL
        return True

# Function that simulate X number of turns as per request
    def complete_sim(self):
        try:
            nb_t=int(self.nb_sim.get())
        except:
            return
        still_fight=True        
        for i in range(nb_t):
            still_fight=self.next_T()
            if not still_fight: break
        if still_fight: self.stalemate(nb_t)

#### ALL functions raising POP UPs are below
    # Function that raise a pop up to show which team quantity is not correct    
    def wrong_quantity(self,type,team):
        x,y = self.master.winfo_x(), self.master.winfo_y()     
        Wrong_Q = tk.Toplevel()		  # Popup -> Toplevel()
        Wrong_Q.geometry("+%d+%d" % (x + 200, y + 300))
        Wrong_Q.title('Incorrect number of Robots')
        mess_WQ='{} has {} Robots'.format(team,['zero','more than {} (size of the board)'.format(size_board)][type])
        if team=='Red Team':
            L_Wrong_Q=tk.Label(Wrong_Q,text=mess_WQ,font=('Calibri',14),fg="red") 
        else:
            L_Wrong_Q=tk.Label(Wrong_Q,text=mess_WQ,font=('Calibri',14),fg="blue")              
        L_Wrong_Q.pack()
        tk.Button(Wrong_Q, text='OK', command=Wrong_Q.destroy).pack(padx=10, pady=10)        
        Wrong_Q.transient(self.master) 	  
        Wrong_Q.grab_set()		  
        self.master.wait_window(Wrong_Q)  
    
    # Function that raise a pop up to confirm the user wants to launch a specific number of turns
    def Complete_popup(self): 
        x,y = self.master.winfo_x(), self.master.winfo_y()           
        Sim_conf=tk.Toplevel(bg='white')
        Sim_conf.geometry("+%d+%d" % (x + 200, y + 300))
        Sim_conf.title("Simulate the fight until the end")
        L_conf=tk.Label(Sim_conf,text="Click OK to confirm the simulation of {} turns".format(self.nb_sim.get()), bg='white', font=("Calibri",14),pady=10,padx=20)
        L_conf.pack()                
        L_conf2=tk.Label(Sim_conf,text="If all the robots of either team are destroyed, the simulation will stop",bg='white',padx=10,pady=15,font=('Calibri',14))
        L_conf2.pack()
        
        tk.Button(Sim_conf, text='OK', command=lambda:[Sim_conf.destroy(),self.complete_sim()]).pack(side = tk.LEFT,padx=10, pady=10)
        tk.Button(Sim_conf, text='Cancel', command=Sim_conf.destroy).pack(side = tk.LEFT,padx=10, pady=10)
        Sim_conf.transient(self.master)
        Sim_conf.grab_set()
        self.master.wait_window(Sim_conf)


# Function that raise a pop up when the number of requested simulated turns is done
    def stalemate(self,nb_t):
        x,y = self.master.winfo_x(), self.master.winfo_y()    
        SM=tk.Toplevel()
        SM.geometry("+%d+%d" % (x + 200, y + 300))
        SM.title("End of Simulation")
        L_SM=tk.Label(SM,text="After the {} turns, no team has won\n {} Red Robot(s) and {} Blue Robot(s) are still active".format(nb_t,len(red_team),len(blue_team)),font=('Calibri',14))
        L_SM.pack(padx=10,pady=10)
        tk.Button(SM,text='OK',command=SM.destroy).pack(padx=10,pady=10)
        self.master.wait_window(SM)    

    def Victory_popup(self,vic_T,lose_T):
        x,y = self.master.winfo_x(), self.master.winfo_y()    
        Vic = tk.Toplevel()	
        Vic.geometry("+%d+%d" % (x + 200, y + 300))	 
        Vic.title('End of the Game')
        mess_Vic1='All robots of the {} have been destroyed'.format(lose_T)
        mess_Vic2='The {} wins!!'.format(vic_T)
        L_Vic1=tk.Label(Vic,text=mess_Vic1,font=('Calibri',16))
        if vic_T=='Red Team':
            L_Vic2=tk.Label(Vic,text=mess_Vic2,fg='red',font=('Calibri',18))
        else:
            L_Vic2=tk.Label(Vic,text=mess_Vic2,fg='blue',font=('Calibri',18))
        L_Vic1.pack(pady=5,padx=10)
        L_Vic2.pack()
        tk.Button(Vic, text='OK', command=Vic.destroy).pack(padx=10, pady=10)        
        Vic.transient(self.master) 	  
        Vic.grab_set()		  
        self.master.wait_window(Vic)   
    
### Launching the application
if __name__ == '__main__':
   root = tk.Tk()
   main_app =  MainApplication(root)
   root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\je2f_\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-25-27442d5555c8>", line 490, in <lambda>
    tk.Button(Sim_conf, text='OK', command=lambda:[Sim_conf.destroy(),self.complete_sim()]).pack(side = tk.LEFT,padx=10, pady=10)
  File "<ipython-input-25-27442d5555c8>", line 454, in complete_sim
    nb_t=int(self.nb_sim.get())
ValueError: invalid literal for int() with base 10: ''


In [ ]:
README
======

VOILA
-------------